In [1]:
import os
from ultralytics import NAS
os.chdir("..")
ROOT = os.getcwd()
print(ROOT)


/Users/mautushid/github/Cowsformer


In [2]:
from models.nas import *


[2024-03-03 14:26:37] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /Users/mautushid/sg_logs/console.log


[2024-03-03 14:26:37] WARNING - __init__.py - Failed to import pytorch_quantization
[2024-03-03 14:26:38] WARNING - redirects.py - NOTE: Redirects are currently not supported in Windows or MacOs.
[2024-03-03 14:26:39] WARNING - calibrator.py - Failed to import pytorch_quantization
[2024-03-03 14:26:39] WARNING - export.py - Failed to import pytorch_quantization
[2024-03-03 14:26:39] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2024-03-03 14:26:39] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernels. Some features may not work as expected.


In [3]:
### class inits and other inputs

path_model = 'yolo_nas_l' 
dir_train = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/train"
dir_val = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/val"
dir_test = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test_old"
name_task = "cow200"

data_yaml_path = "/Users/mautushid/github/Cowsformer/data/cow200/yolov5/data.yaml"
finetuned_model_path = "/Users/mautushid/github/Cowsformer/__notebooks/lms_checkpoints/yolo_l_100/n100_yolo_na_i2_exp_yolo_nas_l_100_2/ckpt_best.pth"

### Creating instance of Niche_YOLO_NAS class
my_nas = Niche_YOLO_NAS(path_model, dir_train, dir_val, dir_test, name_task)

[2024-03-03 14:26:39] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-03-03 14:26:40] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


In [4]:
### load finetuned model
model = my_nas.load(path_model,finetuned_model_path)

[2024-03-03 14:26:41] INFO - checkpoint_utils.py - Successfully loaded model weights from /Users/mautushid/github/Cowsformer/__notebooks/lms_checkpoints/yolo_l_100/n100_yolo_na_i2_exp_yolo_nas_l_100_2/ckpt_best.pth EMA checkpoint.


In [6]:
import supervision as sv

ds = sv.DetectionDataset.from_yolo(
    images_directory_path="/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test_old/images",
    annotations_directory_path="/Users/mautushid/github/Cowsformer/data/cow200/yolov5/test_old/labels",
    data_yaml_path="/Users/mautushid/github/Cowsformer/data/cow200/yolov5/data.yaml",
    force_masks=False
)
CONFIDENCE_TRESHOLD = 0.5

predictions = {}

for image_name, image in ds.images.items():
    result = list(model.predict(
        image, conf=CONFIDENCE_TRESHOLD))[0]
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        #xyxy=result.prediction.bboxes_xywh,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections

/Users/mautushid/miniconda3/envs/myenv/lib/python3.9/site-packages/numpy/lib/arraypad.py:487: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)
/Users/mautushid/miniconda3/envs/myenv/lib/python3.9/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
[2024-03-03 14:27:06] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-03-03 14:27:06] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-03-03 14:27

In [57]:
import os
'''
# Assuming `predictions` is your dictionary of results as shown above
# And assuming `output_directory` is the path to the directory where you want to save the .txt files
output_directory = 'path/to/output/directory'

# Ensure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate over the predictions and write them to files
for image_name, detections in predictions.items():
    # Prepare the output file path
    base_filename = os.path.splitext(image_name)[0]
    output_file_path = os.path.join(output_directory, base_filename + '.txt')

    # Open the file and write the detections
    with open(output_file_path, 'w') as file:
        for bbox, conf, class_id in zip(detections.xyxy, detections.confidence, detections.class_id):
            # Create a string for each detection
            # Format: class_id x_min y_min x_max y_max confidence
            detection_str = f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} {conf}\n"
            # Write to file
            file.write(detection_str)

print(f"All predictions have been written to {output_directory}")
'''

All predictions have been written to path/to/output/directory


In [76]:
############################################################
# The following methods are not used in the current project
############################################################
    

In [7]:
from API import*
api = YOLO_API("/Users/mautushid/github/Cowsformer/data/cow200/yolov5") 
lbs = api.get_gt_detections(
    "test_old")
pre = api.get_pred_detections(
    "test_old/output")

test_old
test_old/output


In [8]:
from evaluate import from_sv

In [9]:
op = from_sv(pre, lbs)

In [10]:
print(op)

{'map5095': 0.7202700813553288, 'map50': 0.9558591094378901, 'precision': 0.854066985645933, 'recall': 0.9444444444444444, 'f1': 0.8969849246231155, 'n_all': 378, 'n_fn': 21, 'n_fp': 61}


In [11]:
import pandas as pd


def make_table_row(EvalOutput, dataset_name, model_name, data_size):
    # Combine EvalOutput with the additional information
    combined_dict = {
        **EvalOutput,  # Unpack the EvalOutput dictionary
        'config': dataset_name,  # Add dataset_name as 'config'
        'model': model_name,  # Add model_name as 'model'
        'n': data_size  # Add data_size as 'n'
    }

    # Create a DataFrame from the combined dictionary
    # The index=[0] makes the dictionary values to be considered as a single row
    df = pd.DataFrame([combined_dict])

    return df



dataset_name = "cow200"
model_name = "yolo_nas_l"
data_size = "200"

df = make_table_row(op, dataset_name, model_name, data_size)

# Display the DataFrame
print(df)

   map5095     map50  precision    recall        f1  n_all  n_fn  n_fp   
0  0.72027  0.955859   0.854067  0.944444  0.896985    378    21    61  \

   config       model    n  
0  cow200  yolo_nas_l  200  


In [12]:
df

,map5095,map50,precision,recall,f1,n_all,n_fn,n_fp,config,model,n
0,0.72027,0.955859,0.854067,0.944444,0.896985,378,21,61,cow200,yolo_nas_l,200
